In [1]:
## This file implements neural networks and random forest on p0040presabsSTCC_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 56.32% for over-sampling data, 62.07% after improvement.
## And it is 93.33% for combination data, and is 90% after regularization.
## For random forest, the accuracy is 58.62% for over-sampling data and 86.67% for combination data.
## For random forest with cross-validation, the mean accuracy is 60.29% for over-sampling, and 97.13% for combination data.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0040presabsSTCC_qual.csv')
df.shape

(255, 375)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
5      2
6      2
7      0
8      1
9      1
10     0
11     2
12     1
13     0
14     2
15     2
16     0
17     0
18     1
19     0
20     0
21     0
22     0
23     2
24     2
25     0
26     2
27     0
28     1
29     2
      ..
225    2
226    1
227    1
228    2
229    2
230    1
231    1
232    2
233    1
234    1
235    2
236    1
237    2
238    1
239    0
240    1
241    1
242    0
243    1
244    1
245    2
246    2
247    1
248    2
249    1
250    2
251    1
252    1
253    2
254    2
Name: pheno, Length: 255, dtype: int64

In [6]:
df['pheno'].value_counts()

2    96
1    95
0    64
Name: pheno, dtype: int64

In [7]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,...,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,ST,CC,pheno
0,107,1,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,2
1,109,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,8,8,0
2,115,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
3,120335,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,120337,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 374)

In [10]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,ST,CC,pheno
0,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,5,5,2
1,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,8,8,0
2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
3,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [11]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 373) (255,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 96), (1, 96), (2, 96)]


Using TensorFlow backend.


In [13]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 46), (1, 10), (2, 42)]


In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
############# Fully-Connected Neural Network ################

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 201 samples, validate on 87 samples
Epoch 1/100
201/201 [==============================] - 0s 600us/step - loss: 7.0540 - accuracy: 0.4229 - val_loss: 2.5152 - val_accuracy: 0.4368
Epoch 2/100
201/201 [==============================] - 0s 102us/step - loss: 4.4470 - accuracy: 0.4776 - val_loss: 1.8173 - val_accuracy: 0.4828
Epoch 3/100
201/201 [==============================] - 0s 110us/step - loss: 2.3024 - accuracy: 0.5373 - val_loss: 1.3059 - val_accuracy: 0.5057
Epoch 4/100
201/201 [==============================] - 0s 108us/step - loss: 1.1992 - accuracy: 0.5373 - val_loss: 2.4253 - val_accuracy: 0.4483
Epoch 5/100
201/201 [==============================] - 0s 117us/step - loss: 1.2074 - accuracy: 0.5373 - val_loss: 1.8814 - val_accuracy: 0.4598
Epoch 6/100
201/201 [==============================] - 0s 117us/step - loss: 1.1637 - accuracy: 0.5323 - val_loss: 1.1745 - val_accuracy: 0.5172
Epoch 7/100
201/201 [==============================] - 0s 126us/step - loss: 1.2632 -

Epoch 57/100
201/201 [==============================] - 0s 118us/step - loss: 0.4204 - accuracy: 0.8706 - val_loss: 1.2105 - val_accuracy: 0.5632
Epoch 58/100
201/201 [==============================] - 0s 107us/step - loss: 0.4352 - accuracy: 0.8657 - val_loss: 1.3509 - val_accuracy: 0.6092
Epoch 59/100
201/201 [==============================] - 0s 115us/step - loss: 0.4274 - accuracy: 0.8557 - val_loss: 1.3313 - val_accuracy: 0.5402
Epoch 60/100
201/201 [==============================] - 0s 121us/step - loss: 0.3877 - accuracy: 0.8607 - val_loss: 1.2990 - val_accuracy: 0.5747
Epoch 61/100
201/201 [==============================] - 0s 120us/step - loss: 0.3703 - accuracy: 0.8856 - val_loss: 1.2393 - val_accuracy: 0.5862
Epoch 62/100
201/201 [==============================] - 0s 155us/step - loss: 0.3634 - accuracy: 0.8806 - val_loss: 1.2403 - val_accuracy: 0.5632
Epoch 63/100
201/201 [==============================] - 0s 191us/step - loss: 0.3601 - accuracy: 0.8905 - val_loss: 1.2509 -

In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

87/87 [==============================] - 0s 128us/step
over-sampling test accuracy: 56.32%


In [22]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [23]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 201 samples, validate on 87 samples
Epoch 1/1000
201/201 [==============================] - 0s 1ms/step - loss: 7.7140 - accuracy: 0.4229 - val_loss: 5.7387 - val_accuracy: 0.4483
Epoch 2/1000
201/201 [==============================] - 0s 166us/step - loss: 4.0782 - accuracy: 0.4428 - val_loss: 3.3195 - val_accuracy: 0.4368
Epoch 3/1000
201/201 [==============================] - 0s 161us/step - loss: 2.4303 - accuracy: 0.4577 - val_loss: 3.5620 - val_accuracy: 0.4368
Epoch 4/1000
201/201 [==============================] - 0s 155us/step - loss: 1.7229 - accuracy: 0.4726 - val_loss: 1.2236 - val_accuracy: 0.4943
Epoch 5/1000
201/201 [==============================] - 0s 173us/step - loss: 1.1993 - accuracy: 0.5274 - val_loss: 1.1705 - val_accuracy: 0.4253
Epoch 6/1000
201/201 [==============================] - 0s 154us/step - loss: 1.1721 - accuracy: 0.5622 - val_loss: 1.2543 - val_accuracy: 0.4598
Epoch 7/1000
201/201 [==============================] - 0s 146us/step - loss: 1.2

201/201 [==============================] - 0s 151us/step - loss: 0.3962 - accuracy: 0.8756 - val_loss: 1.3168 - val_accuracy: 0.6092
Epoch 57/1000
201/201 [==============================] - 0s 159us/step - loss: 0.3538 - accuracy: 0.8706 - val_loss: 1.3354 - val_accuracy: 0.5977
Epoch 58/1000
201/201 [==============================] - 0s 152us/step - loss: 0.3492 - accuracy: 0.8806 - val_loss: 1.3192 - val_accuracy: 0.5862
Epoch 59/1000
201/201 [==============================] - 0s 124us/step - loss: 0.3636 - accuracy: 0.8756 - val_loss: 1.3381 - val_accuracy: 0.5632
Epoch 60/1000
201/201 [==============================] - 0s 150us/step - loss: 0.3627 - accuracy: 0.8607 - val_loss: 1.3697 - val_accuracy: 0.5977
Epoch 61/1000
201/201 [==============================] - 0s 176us/step - loss: 0.3400 - accuracy: 0.8806 - val_loss: 1.3690 - val_accuracy: 0.6092
Epoch 62/1000
201/201 [==============================] - 0s 140us/step - loss: 0.3195 - accuracy: 0.8806 - val_loss: 1.2946 - val_ac

Epoch 112/1000
201/201 [==============================] - 0s 129us/step - loss: 0.2299 - accuracy: 0.9254 - val_loss: 1.7025 - val_accuracy: 0.5977
Epoch 113/1000
201/201 [==============================] - 0s 127us/step - loss: 0.2190 - accuracy: 0.9254 - val_loss: 1.7707 - val_accuracy: 0.6552
Epoch 114/1000
201/201 [==============================] - 0s 128us/step - loss: 0.2098 - accuracy: 0.9154 - val_loss: 1.7039 - val_accuracy: 0.6207
Epoch 115/1000
201/201 [==============================] - 0s 130us/step - loss: 0.3244 - accuracy: 0.8955 - val_loss: 1.7601 - val_accuracy: 0.6092
Epoch 116/1000
201/201 [==============================] - 0s 100us/step - loss: 0.2773 - accuracy: 0.8955 - val_loss: 1.7510 - val_accuracy: 0.6437
Epoch 117/1000
201/201 [==============================] - 0s 100us/step - loss: 0.2714 - accuracy: 0.9055 - val_loss: 1.7839 - val_accuracy: 0.6207
Epoch 118/1000
201/201 [==============================] - 0s 98us/step - loss: 0.4819 - accuracy: 0.8955 - val_l

201/201 [==============================] - 0s 134us/step - loss: 0.1466 - accuracy: 0.9453 - val_loss: 2.0377 - val_accuracy: 0.6322
Epoch 168/1000
201/201 [==============================] - 0s 157us/step - loss: 0.1562 - accuracy: 0.9502 - val_loss: 2.0446 - val_accuracy: 0.6092
Epoch 169/1000
201/201 [==============================] - 0s 159us/step - loss: 0.1450 - accuracy: 0.9403 - val_loss: 2.0094 - val_accuracy: 0.5977
Epoch 170/1000
201/201 [==============================] - 0s 123us/step - loss: 0.1491 - accuracy: 0.9303 - val_loss: 2.0936 - val_accuracy: 0.6207
Epoch 171/1000
201/201 [==============================] - 0s 182us/step - loss: 0.1941 - accuracy: 0.9403 - val_loss: 2.1761 - val_accuracy: 0.5862
Epoch 172/1000
201/201 [==============================] - 0s 184us/step - loss: 0.1735 - accuracy: 0.9254 - val_loss: 2.0645 - val_accuracy: 0.6092
Epoch 173/1000
201/201 [==============================] - 0s 122us/step - loss: 0.1473 - accuracy: 0.9403 - val_loss: 2.0814 - 

201/201 [==============================] - 0s 115us/step - loss: 0.1209 - accuracy: 0.9652 - val_loss: 2.3341 - val_accuracy: 0.6552
Epoch 223/1000
201/201 [==============================] - 0s 88us/step - loss: 0.1120 - accuracy: 0.9652 - val_loss: 2.4677 - val_accuracy: 0.6782
Epoch 224/1000
201/201 [==============================] - 0s 87us/step - loss: 0.1073 - accuracy: 0.9751 - val_loss: 2.3717 - val_accuracy: 0.6437
Epoch 225/1000
201/201 [==============================] - 0s 84us/step - loss: 0.1074 - accuracy: 0.9701 - val_loss: 2.3769 - val_accuracy: 0.6552
Epoch 226/1000
201/201 [==============================] - 0s 96us/step - loss: 0.1079 - accuracy: 0.9602 - val_loss: 2.3934 - val_accuracy: 0.6552
Epoch 227/1000
201/201 [==============================] - 0s 82us/step - loss: 0.1084 - accuracy: 0.9701 - val_loss: 2.3706 - val_accuracy: 0.6437
Epoch 228/1000
201/201 [==============================] - 0s 132us/step - loss: 0.1544 - accuracy: 0.9602 - val_loss: 2.4455 - val_a

...

201/201 [==============================] - 0s 93us/step - loss: 0.0247 - accuracy: 0.9900 - val_loss: 4.9178 - val_accuracy: 0.6207
Epoch 774/1000
201/201 [==============================] - 0s 94us/step - loss: 0.0262 - accuracy: 0.9900 - val_loss: 4.9164 - val_accuracy: 0.6322
Epoch 775/1000
201/201 [==============================] - 0s 93us/step - loss: 0.0256 - accuracy: 0.9851 - val_loss: 4.9873 - val_accuracy: 0.6092
Epoch 776/1000
201/201 [==============================] - 0s 100us/step - loss: 0.0223 - accuracy: 0.9900 - val_loss: 5.0346 - val_accuracy: 0.6207
Epoch 777/1000
201/201 [==============================] - 0s 101us/step - loss: 0.0258 - accuracy: 0.9900 - val_loss: 5.0120 - val_accuracy: 0.6092
Epoch 778/1000
201/201 [==============================] - 0s 89us/step - loss: 0.0242 - accuracy: 0.9851 - val_loss: 5.0099 - val_accuracy: 0.6092
Epoch 779/1000
201/201 [==============================] - 0s 88us/step - loss: 0.0251 - accuracy: 0.9851 - val_loss: 5.0584 - val_a

Epoch 829/1000
201/201 [==============================] - 0s 86us/step - loss: 0.0833 - accuracy: 0.9801 - val_loss: 5.1738 - val_accuracy: 0.5862
Epoch 830/1000
201/201 [==============================] - 0s 85us/step - loss: 0.0921 - accuracy: 0.9751 - val_loss: 4.9774 - val_accuracy: 0.5977
Epoch 831/1000
201/201 [==============================] - 0s 80us/step - loss: 0.0318 - accuracy: 0.9851 - val_loss: 5.0143 - val_accuracy: 0.6092
Epoch 832/1000
201/201 [==============================] - 0s 78us/step - loss: 0.0351 - accuracy: 0.9851 - val_loss: 5.0897 - val_accuracy: 0.6207
Epoch 833/1000
201/201 [==============================] - 0s 82us/step - loss: 0.0266 - accuracy: 0.9900 - val_loss: 5.1979 - val_accuracy: 0.6092
Epoch 834/1000
201/201 [==============================] - 0s 91us/step - loss: 0.0263 - accuracy: 0.9851 - val_loss: 5.2136 - val_accuracy: 0.6092
Epoch 835/1000
201/201 [==============================] - 0s 94us/step - loss: 0.0293 - accuracy: 0.9900 - val_loss: 5

Epoch 885/1000
201/201 [==============================] - 0s 95us/step - loss: 0.0260 - accuracy: 0.9900 - val_loss: 5.2540 - val_accuracy: 0.6207
Epoch 886/1000
201/201 [==============================] - 0s 99us/step - loss: 0.0273 - accuracy: 0.9900 - val_loss: 5.1988 - val_accuracy: 0.6092
Epoch 887/1000
201/201 [==============================] - 0s 98us/step - loss: 0.0284 - accuracy: 0.9900 - val_loss: 5.2738 - val_accuracy: 0.6092
Epoch 888/1000
201/201 [==============================] - 0s 86us/step - loss: 0.0235 - accuracy: 0.9851 - val_loss: 5.3141 - val_accuracy: 0.6207
Epoch 889/1000
201/201 [==============================] - 0s 93us/step - loss: 0.0237 - accuracy: 0.9900 - val_loss: 5.3051 - val_accuracy: 0.6092
Epoch 890/1000
201/201 [==============================] - 0s 97us/step - loss: 0.0211 - accuracy: 0.9851 - val_loss: 5.3174 - val_accuracy: 0.6092
Epoch 891/1000
201/201 [==============================] - 0s 86us/step - loss: 0.0240 - accuracy: 0.9851 - val_loss: 5

201/201 [==============================] - 0s 91us/step - loss: 0.0250 - accuracy: 0.9900 - val_loss: 4.8519 - val_accuracy: 0.5977
Epoch 941/1000
201/201 [==============================] - 0s 90us/step - loss: 0.0263 - accuracy: 0.9900 - val_loss: 4.8402 - val_accuracy: 0.5977
Epoch 942/1000
201/201 [==============================] - 0s 85us/step - loss: 0.0239 - accuracy: 0.9900 - val_loss: 4.8746 - val_accuracy: 0.5977
Epoch 943/1000
201/201 [==============================] - 0s 83us/step - loss: 0.0239 - accuracy: 0.9900 - val_loss: 4.8652 - val_accuracy: 0.5977
Epoch 944/1000
201/201 [==============================] - 0s 101us/step - loss: 0.0250 - accuracy: 0.9900 - val_loss: 4.8934 - val_accuracy: 0.5977
Epoch 945/1000
201/201 [==============================] - 0s 95us/step - loss: 0.0242 - accuracy: 0.9900 - val_loss: 4.8641 - val_accuracy: 0.5977
Epoch 946/1000
201/201 [==============================] - 0s 99us/step - loss: 0.0241 - accuracy: 0.9900 - val_loss: 4.9115 - val_ac

Epoch 996/1000
201/201 [==============================] - 0s 92us/step - loss: 0.0287 - accuracy: 0.9801 - val_loss: 5.1091 - val_accuracy: 0.6207
Epoch 997/1000
201/201 [==============================] - 0s 87us/step - loss: 0.0315 - accuracy: 0.9900 - val_loss: 4.9599 - val_accuracy: 0.6207
Epoch 998/1000
201/201 [==============================] - 0s 84us/step - loss: 0.0306 - accuracy: 0.9900 - val_loss: 4.9437 - val_accuracy: 0.6207
Epoch 999/1000
201/201 [==============================] - 0s 93us/step - loss: 0.0264 - accuracy: 0.9851 - val_loss: 5.2041 - val_accuracy: 0.6207
Epoch 1000/1000
201/201 [==============================] - 0s 110us/step - loss: 0.0416 - accuracy: 0.9801 - val_loss: 5.0793 - val_accuracy: 0.6207


In [25]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

87/87 [==============================] - 0s 91us/step
over-sampling test accuracy: 62.07%


In [26]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [27]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 68 samples, validate on 30 samples
Epoch 1/100
68/68 [==============================] - 0s 1ms/step - loss: 5.1253 - accuracy: 0.3676 - val_loss: 10.3768 - val_accuracy: 0.5333
Epoch 2/100
68/68 [==============================] - 0s 163us/step - loss: 4.1137 - accuracy: 0.6765 - val_loss: 8.7989 - val_accuracy: 0.7000
Epoch 3/100
68/68 [==============================] - 0s 150us/step - loss: 3.3963 - accuracy: 0.7353 - val_loss: 7.3656 - val_accuracy: 0.7000
Epoch 4/100
68/68 [==============================] - 0s 143us/step - loss: 2.7550 - accuracy: 0.7500 - val_loss: 6.0373 - val_accuracy: 0.7000
Epoch 5/100
68/68 [==============================] - 0s 149us/step - loss: 2.2463 - accuracy: 0.7647 - val_loss: 4.7772 - val_accuracy: 0.7000
Epoch 6/100
68/68 [==============================] - 0s 156us/step - loss: 1.7263 - accuracy: 0.7794 - val_loss: 3.5860 - val_accuracy: 0.7000
Epoch 7/100
68/68 [==============================] - 0s 144us/step - loss: 1.2826 - accuracy: 0.794

68/68 [==============================] - 0s 153us/step - loss: 0.0835 - accuracy: 0.9412 - val_loss: 0.1959 - val_accuracy: 0.9333
Epoch 58/100
68/68 [==============================] - 0s 175us/step - loss: 0.0712 - accuracy: 1.0000 - val_loss: 0.1611 - val_accuracy: 0.9667
Epoch 59/100
68/68 [==============================] - 0s 154us/step - loss: 0.0609 - accuracy: 1.0000 - val_loss: 0.1579 - val_accuracy: 0.9333
Epoch 60/100
68/68 [==============================] - 0s 177us/step - loss: 0.0664 - accuracy: 0.9853 - val_loss: 0.1625 - val_accuracy: 0.9667
Epoch 61/100
68/68 [==============================] - 0s 165us/step - loss: 0.0608 - accuracy: 0.9853 - val_loss: 0.2794 - val_accuracy: 0.9333
Epoch 62/100
68/68 [==============================] - 0s 114us/step - loss: 0.0671 - accuracy: 1.0000 - val_loss: 0.3965 - val_accuracy: 0.8000
Epoch 63/100
68/68 [==============================] - 0s 136us/step - loss: 0.0746 - accuracy: 0.9853 - val_loss: 0.4202 - val_accuracy: 0.8000
Epoch

In [29]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

30/30 [==============================] - 0s 122us/step
combination test accuracy: 93.33%


In [30]:
#### add dropout and regularizor
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [31]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 68 samples, validate on 30 samples
Epoch 1/100
68/68 [==============================] - 0s 3ms/step - loss: 7.2758 - accuracy: 0.2794 - val_loss: 8.1717 - val_accuracy: 0.5000
Epoch 2/100
68/68 [==============================] - 0s 142us/step - loss: 7.2876 - accuracy: 0.4706 - val_loss: 7.5995 - val_accuracy: 0.6333
Epoch 3/100
68/68 [==============================] - 0s 160us/step - loss: 5.6766 - accuracy: 0.6324 - val_loss: 7.5942 - val_accuracy: 0.7000
Epoch 4/100
68/68 [==============================] - 0s 192us/step - loss: 4.8370 - accuracy: 0.7059 - val_loss: 7.0370 - val_accuracy: 0.9000
Epoch 5/100
68/68 [==============================] - 0s 174us/step - loss: 6.5928 - accuracy: 0.6912 - val_loss: 6.9261 - val_accuracy: 0.9000
Epoch 6/100
68/68 [==============================] - 0s 185us/step - loss: 4.8857 - accuracy: 0.7353 - val_loss: 6.7402 - val_accuracy: 0.9000
Epoch 7/100
68/68 [==============================] - 0s 167us/step - loss: 5.5025 - accuracy: 0.6912

Epoch 58/100
68/68 [==============================] - 0s 209us/step - loss: 2.6819 - accuracy: 0.7941 - val_loss: 3.5949 - val_accuracy: 0.9000
Epoch 59/100
68/68 [==============================] - 0s 183us/step - loss: 2.3295 - accuracy: 0.8088 - val_loss: 3.3675 - val_accuracy: 0.9000
Epoch 60/100
68/68 [==============================] - 0s 175us/step - loss: 2.5542 - accuracy: 0.7647 - val_loss: 3.1941 - val_accuracy: 0.9000
Epoch 61/100
68/68 [==============================] - 0s 219us/step - loss: 2.0893 - accuracy: 0.7794 - val_loss: 3.1294 - val_accuracy: 0.9000
Epoch 62/100
68/68 [==============================] - 0s 213us/step - loss: 2.4091 - accuracy: 0.7500 - val_loss: 3.0982 - val_accuracy: 0.9000
Epoch 63/100
68/68 [==============================] - 0s 216us/step - loss: 2.3642 - accuracy: 0.7794 - val_loss: 3.0940 - val_accuracy: 0.9000
Epoch 64/100
68/68 [==============================] - 0s 229us/step - loss: 2.2401 - accuracy: 0.7941 - val_loss: 3.0902 - val_accuracy:

In [33]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

30/30 [==============================] - 0s 54us/step
combination test accuracy: 93.33%


In [34]:
############## Random Forest ##############

In [35]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [36]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 58.62%


In [37]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [38]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 86.67%


In [39]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.5        0.64285714 0.58974359 0.61538462 0.66666667]
0.6029304029304029


In [40]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         0.93333333 0.92307692 1.         1.        ]
0.9712820512820514
